In [1]:
# Parameters
NUM_AGENTS = 5
NUM_MEETINGS = 15
AGENT_SKILLS = ['fire_cert', 'bodyguard_cert', 'first_aid', 'security_clearance']
NIGHT_SHIFTS_PER_AGENT_PER_WEEK = 1
AGENT_HOURS_PER_WEEK = 40
MEETING_DURATION_OPTIONS = [1, 2, 3]  # In hours


In [2]:
import random

# Define the zones
zones = [
    {'sw': (46.160963, 6.084271), 'ne': (46.192016, 6.138650)},  # GENEVA1
    {'sw': (46.213758, 6.059573), 'ne': (46.235041, 6.144283)},  # GENEVA2
    {'sw': (46.190981, 6.144247), 'ne': (46.205087, 6.183324)},  # GENEVA3
    {'sw': (46.160963, 6.0),      'ne': (46.192016, 6.06)}       # GENEVA4
]

def generate_random_location(zones):
    zone = random.choice(zones)
    lat = random.uniform(zone['sw'][0], zone['ne'][0])
    lon = random.uniform(zone['sw'][1], zone['ne'][1])
    return (lat, lon)



In [3]:
def generate_agents(num_agents):
    agents = []
    for i in range(num_agents):
        agent = {
            'agent_id': f'Agent_{i+1}',
            'skills': random.sample(AGENT_SKILLS, random.randint(1, len(AGENT_SKILLS))),
            'home_location': generate_random_location(zones),
            'night_shifts_per_week': NIGHT_SHIFTS_PER_AGENT_PER_WEEK,
            'weekly_hours': AGENT_HOURS_PER_WEEK
        }
        agents.append(agent)
    return agents

def generate_meetings(num_meetings):
    meetings = []
    for i in range(num_meetings):
        meeting_duration = random.choice(MEETING_DURATION_OPTIONS)
        meeting = {
            'meeting_id': f'Meeting_{i+1}',
            'required_skill': random.choice(AGENT_SKILLS),
            'location': generate_random_location(zones),
            'time_window': (
                random.randint(8, 16),  # Start hour
                random.randint(9, 18)   # End hour
            ),
            'duration': meeting_duration
        }
        # Ensure the time window makes sense
        if meeting['time_window'][1] - meeting['time_window'][0] < meeting_duration:
            meeting['time_window'] = (
                meeting['time_window'][0],
                meeting['time_window'][0] + meeting_duration
            )
        meetings.append(meeting)
    return meetings


In [5]:
# Initialize agents and meetings
agents = generate_agents(NUM_AGENTS)
meetings = generate_meetings(NUM_MEETINGS)


In [6]:
import folium

# Create a map centered around Geneva
map_geneva = folium.Map(location=[46.2044, 6.1432], zoom_start=13)

# Add agent home locations
for agent in agents:
    folium.Marker(
        location=agent['home_location'],
        popup=f"{agent['agent_id']} Skills: {', '.join(agent['skills'])}",
        icon=folium.Icon(color='blue', icon='user')
    ).add_to(map_geneva)

# Add meeting locations
for meeting in meetings:
    folium.Marker(
        location=meeting['location'],
        popup=f"{meeting['meeting_id']} Required Skill: {meeting['required_skill']}",
        icon=folium.Icon(color='red', icon='briefcase')
    ).add_to(map_geneva)

map_geneva


In [6]:
import json

# Prepare data for Timefold
timefold_data = {
    'vehicles': [
        {
            'name': agent['agent_id'],
            'homeLocation': agent['home_location'],
            'capacity': agent['weekly_hours'],
            'skills': agent['skills']
        }
        for agent in agents
    ],
    'visits': [
        {
            'name': meeting['meeting_id'],
            'location': meeting['location'],
            'required_skill': meeting['required_skill'],
            'time_window': meeting['time_window'],
            'duration': meeting['duration']
        }
        for meeting in meetings
    ]
}

# Save to JSON file with indentation
with open('timefold-data.json', 'w') as f:
    json.dump(timefold_data, f, indent=4)  # Added indent parameter
